In [ ]:
import numpy as np
import matplotlib.pyplot as plot
from scipy import interpolate

In [ ]:
chrisRawData = np.loadtxt("supernovadmplot.txt", skiprows=0, dtype="str")
chrisData = {}
for i in range(0,len(chrisRawData),2):
    chrisData[chrisRawData[i].lower()] = np.array(chrisRawData[i+1].split(",")).astype(float)

In [ ]:
myRawData = np.loadtxt("Oper_c1-0_IsotopicBreakdown.dat", skiprows=2, dtype="str")
isotopes = ["hydrogen", "helium", "carbon", "oxygen", "neon", "magnesium", "silicon", "sulfur", "iron"]
#   Change the length the the number of velocity points run + number of comments between each velocity (eg do 1,1001  +  print*, "isotope ", eli  ->  length=1002)
length = 10002
myData = {}
for i in range(len(isotopes)):
    isotopicData = []
    for j in range(1,length):
        isotopicData.append(myRawData[i*length+j][2])
    myData[isotopes[i].lower()] = np.array(isotopicData).astype(float)
vel = []
for i in range(1,length):
    vel.append(myRawData[i][1])
myData["velocity"] = np.array(vel).astype(float)

In [ ]:
#   Some useful lists for plotting style and labels and limits 
colours = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "yellow"]
isoMass = [1., 4., 12., 16., 20., 24., 28., 32., 56.]
maxima = [4365, 4435, 4470, 4480, 4480, 4485, 4485, 4485, 4490]

In [ ]:
plot.clf()
ax = plot.subplot(111)

totalChris = np.zeros(chrisData["velocity"].shape)
totalSuper = np.zeros(myData["velocity"].shape)
for i in range(len(isotopes)):
    #   Collect the totals
    totalChris = totalChris + chrisData[isotopes[i]]
    totalSuper = totalSuper + myData[isotopes[i]]
    # if i == 0:
    #   Plot the individual isotopes
    plot.plot(chrisData["velocity"]*10**-5, chrisData[isotopes[i]], label="Chris "+isotopes[i], color=colours[i], linestyle="solid")
    plot.plot(myData["velocity"], myData[isotopes[i]], label="Super "+isotopes[i], color=colours[i], linestyle="dashed")

#   Plot the totals
plot.plot(chrisData["velocity"]*10**-5, totalChris, label="Chris Total", color="black", linestyle="solid")
plot.plot(myData["velocity"], totalSuper, label="Super Total", color="black", linestyle="dashed")

#   Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
#   Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plot.title(r"Isotopic velocity spectrum for $c_{1}^{0}$")
plot.xlabel(r"Dark Matter Velocity [km s$^{-1}$]")
plot.ylabel(r"Scattered Dark Matter[(cm s$^{-1}$)$^{-1}$ s$^{-1}$ cm$^{-2}$]")
# plot.xscale("log")
# plot.yscale("log")
plot.xlim(4300, 4500)
plot.grid()
plot.show()
# plot.savefig("isotopicBreakdown+total.pdf")

In [ ]:
plot.clf()
ax = plot.subplot(111)
totalChris = np.zeros(chrisData["velocity"].shape)
totalSuper = np.zeros(myData["velocity"].shape)

for i in range(len(isotopes)):
    #   Get the totals
    totalChris = totalChris + chrisData[isotopes[i]]
    totalSuper = totalSuper + myData[isotopes[i]]

    #   Get the interpolation (index using Chris' velocity points, which are in cm/s)
    chrisInterp = interpolate.interp1d(chrisData["velocity"]*10**-5, chrisData[isotopes[i]])(chrisData["velocity"]*10**-5)
    superInterp = interpolate.interp1d(myData["velocity"], myData[isotopes[i]])(chrisData["velocity"]*10**-5)

    #   Just for replotting the interpolations
    # plot.plot(chrisData["velocity"]*10**-5, chrisInterp, label="Chris "+isotopes[i], color=colours[i], linestyle="solid")
    # plot.plot(chrisData["velocity"]*10**-5, superInterp, label="Super "+isotopes[i], color=colours[i], linestyle="dashed")

    # if i==0 or i==1:
    #   Plot the relative error as a function of the velocity
    plot.plot(chrisData["velocity"]*10**-5, superInterp/chrisInterp, label=isotopes[i], color=colours[i])

#   Get the total interpolation (index using Chris' velocity points, which are in cm/s)
chrisTotalInterp = interpolate.interp1d(chrisData["velocity"]*10**-5, totalChris)(chrisData["velocity"]*10**-5)
superTotalInterp = interpolate.interp1d(myData["velocity"], totalSuper)(chrisData["velocity"]*10**-5)

#   Just for replotting the total interpolations
# plot.plot(chrisData["velocity"]*10**-5, chrisTotalInterp, label="Chris Total", color="black", linestyle="solid")
# plot.plot(chrisData["velocity"]*10**-5, superTotalInterp, label="Super Total", color="black", linestyle="dashed")

#   Plot the relative error of the total as a function of the velocity
plot.plot(chrisData["velocity"]*10**-5, superTotalInterp/chrisTotalInterp, label="Total", color="black")

#   Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
#   Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plot.title(r"Isotopic velocity spectrum for $c_{1}^{0}$ ratio Super/Chris")
plot.xlabel(r"Dark Matter Velocity [km s$^{-1}$]")
plot.ylabel(r"Scattered Dark Matter[(cm s$^{-1}$)$^{-1}$ s$^{-1}$ cm$^{-2}$]")
# plot.xscale("log")
plot.yscale("log")
# plot.ylim(top=1.0)
plot.xlim(4300, 4500)
plot.grid()
plot.show()
# plot.savefig("isotopicBreakdownRatio+total.pdf")

In [ ]:
plot.clf()
ax = plot.subplot(111)
totalChris = np.zeros(chrisData["velocity"].shape)
totalSuper = np.zeros(myData["velocity"].shape)

print("Ratios as Super/Chris:")
for i in range(len(isotopes)):
    #   Grab the section of the ratio nearest the end (flattest for better average) 
    maximum = maxima[i]
    velArray = np.linspace(maximum-30,maximum,100)#myData["velocity"]

    #   Get the interpolation
    superIso = interpolate.interp1d(myData["velocity"], myData[isotopes[i]])(velArray)
    chrisIso = interpolate.interp1d(chrisData["velocity"]*10**-5, chrisData[isotopes[i]])(velArray)
    currRatio = superIso/chrisIso
    avg = np.average(currRatio)

    #   Just for replotting the interpolation over velArray
    # plot.plot(velArray, superIso, label="Super "+isotopes[i], color=colours[i], linestyle="dashed")

    #   Plotting the relative error by isotopic mass
    plot.scatter(isoMass[i], 1/avg, label=isotopes[i], color=colours[i])
    print(isotopes[i], avg)

#   Trying to model the mass dependent error
massRange = np.linspace(isoMass[0],isoMass[-1],100)
constant = 1
AMU = 0.938
mDM = 1 # assume dark matter mass is 1 GeV
x = massRange*AMU # x is the atomic mass
mu = mDM*x/(mDM+x)
plotMe = np.exp(-mu*5)
constant = 1/0.021112161494109175/plotMe[0]
print(constant)
# plot.plot(massRange, constant*plotMe)#np.sqrt(x)/16)

#   Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
#   Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plot.title(r"Ratio Super/Chris By Atomic Mass")
plot.xlabel(r"Atomic Mass")
plot.ylabel(r"Ratio Super/Chris")
# plot.xscale("log")
# plot.yscale("log")
# plot.ylim(top=1.0)
# plot.xlim(4300, 4500)
plot.grid()
plot.show()
# plot.savefig("temp.pdf")